# villaInventorySdk
[docs](https://thanakijwanavit.github.io/villa-inventory-sdk/)
> read and write inventory in real time

full docs here https://thanakijwanavit.github.io/villa-inventory-sdk/

## Install

`pip install villaInventorySdk`

## How to use

Uploading a large amount of data

In [ ]:
#hide
# isAws = True
# if isAws:
#   USER = None
#   PW = None
# else:
#   import pickle
#   KEY = ''
#   PW = ''
#   keypath = '/Users/nic/.villa-product-master-inventory'
#   if KEY and PW:
#     with open (keypath, 'wb') as f:
#       pickle.dump({
#           'KEY': KEY,
#           'PW': PW
#       }, f)
#   with open(keypath, 'rb') as f:
#     creden = pickle.load(f)
#   USER = creden['KEY']
#   PW = creden['PW']

## sample input

In [ ]:
from villaInventorySdk.inventory import InventorySdk
from random import randrange
import boto3, time, json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from datetime import datetime
import pandas as pd
from nicHelper.dictUtil import printDict

In [ ]:
sampleInput =  [ 
  { 'iprcode': '0000009', 'brcode': '1000', 'ib_cf_qty': '50', 'new_ib_vs_stock_cv': '27' },
  { 'iprcode': '0000004', 'brcode': '1000', 'ib_cf_qty': '35', 'new_ib_vs_stock_cv': '33' },
  { 'iprcode': '0000003', 'brcode': '1003', 'ib_cf_qty': '36', 'new_ib_vs_stock_cv': '33' }
    ]
df = pd.DataFrame(sampleInput)
df

,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,0000009,1000,50,27
1,0000004,1000,35,33
2,0000003,1003,36,33


## Upload data

In [ ]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'
branch = 'dev-manual'

In [ ]:
#hide
inputBucketName= 'input-bucket-test-inventory-6b3753e2f2'
functionName= 'update-inventory-s3-test-inventory-6b3753e2f2'
invocationType = 'RequestResponse'

## init sdk

In [ ]:
%%time
USER=None
PW = None
sdk = InventorySdk(user=USER, pw=PW, branchName = branch)

CPU times: user 34.2 ms, sys: 7.54 ms, total: 41.8 ms
Wall time: 40.8 ms


## Update inventory 

In [ ]:
%%time
key = 'test'
r = sdk.uploadDf(df, key = key)
if r.status_code >= 400: raise Exception(r.json())
sdk.ingestData(key = key)

signed url is 
url : https://in
fields
 key : test
 AWSAccessKeyId : ASIAVX4Z5T
 x-amz-security-token : IQoJb3JpZ2
 policy : eyJleHBpcm
 signature : apy2TqqnY2
CPU times: user 54.7 ms, sys: 12.3 ms, total: 67 ms
Wall time: 628 ms


{'body': '{"iprcode":{"0":"0000009","1":"0000004","2":"0000003"},"brcode":{"0":"1000","1":"1000","2":"1003"},"ib_cf_qty":{"0":"50","1":"35","2":"36"},"new_ib_vs_stock_cv":{"0":"27","1":"33","2":"33"}}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

#### test uploading full data 

In [ ]:
#### test uploading real data 
# df = pd.read_csv('sampleData/inventory.csv', index_col=0, dtype=str).reset_index(drop=True)
# r = sdk.uploadDf(df, key = key)
# if r.status_code >= 400: raise Exception(r.json())
# sdk.ingestData(key = key)

## Query single product

In [ ]:
%%time
sdk.querySingleProduct2(iprcode='1234')

succesfully get url, returning pandas
CPU times: user 103 ms, sys: 12.6 ms, total: 116 ms
Wall time: 799 ms


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv


## Query Branch

In [ ]:
%%time
sdk.branchQuery(brcode='1000', iprcodes = [9])

TypeError: branchQuery() got an unexpected keyword argument 'iprcodes'

## Query All

In [ ]:
%%time
sdk.queryAll2()

succesfully get url, returning pandas
CPU times: user 76.2 ms, sys: 24.1 ms, total: 100 ms
Wall time: 673 ms


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
0,4,1000,35,33
1,9,1000,95,95
2,12,1000,36,36
3,26,1000,28,28
4,28,1000,9,9
...,...,...,...,...
81551,244818,1000,0,0
81552,244820,1000,0,0
81553,244822,1000,0,0
81554,244823,1000,0,0


In [ ]:
sdk.querySingleProduct2()

succesfully get url, returning pandas


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv
